<a href="https://colab.research.google.com/github/DougJ/DougJ.github.io/blob/master/Tensor_From_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create tensors from a CSV file. 
1) Load CSV file as dataframe  
2) Create two unique data frames for single and multi class
3) Export to two unique csv files  
4) Create directories based on unique single class  
5) Copy images from directories into single class target directory   
6) Consider removing or compressing original image file  directories   
7) Explore data, identify if need to adjust prior to tensors   
8) Create tensors   

In [0]:
import pandas as pd
import os

### 1) Load CSV file as dataframe  

In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
cd /content/drive/'My Drive'/images/'NIH chest X ray images'/

/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images


In [138]:
pwd

'/content/drive/.shortcut-targets-by-id/1iVV0f2kJC19GJZlSKvc9A6theRLdsc6-/COVID_19_project/NIH chest X ray images'

In [130]:
ls

Data_Entry_2017.csv  images_004/  images_008/  images_012/
images_001/          images_005/  images_009/  README_CHESTXRAY.pdf
images_002/          images_006/  images_010/
images_003/          images_007/  images_011/


In [131]:
df = pd.DataFrame()
df = pd.read_csv('Data_Entry_2017.csv')
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


### 2) Create two unique data frames for single and multi class

In [0]:
df_pipe = pd.DataFrame(columns=df.columns)
df_noPipe = pd.DataFrame(columns=df.columns)

In [0]:
cols_to_copy = ['Finding Labels', 'Image Index']
# SPECIFIC COLUMNS
df_one_class = df.loc[~df['Finding Labels'].str.contains(r'\|'), cols_to_copy].copy()
df_multi_class = df.loc[df['Finding Labels'].str.contains(r'\|'), cols_to_copy].copy()
# ALL COLUMNS
# df_one_class = df.loc[~df['Finding Labels'].str.contains(r'\|'), :].copy()
#df_multi_class = df.loc[df['Finding Labels'].str.contains(r'\|'), :].copy()

### 3) Export to two unique csv files 

In [0]:
df_one_class.to_csv('Data_Entry_2017_one_class.csv', sep='\t', encoding='utf-8')

In [0]:
df_multi_class.to_csv('Data_Entry_2017_multi_class.csv', sep='\t', encoding='utf-8')

### 4) Create directories based on unique single class  


In [186]:
df_multi_class.head(1)

,Finding Labels,Image Index
1,Cardiomegaly|Emphysema,00000001_001.png


In [187]:
df_one_class.head(1)

,Finding Labels,Image Index
0,Cardiomegaly,00000001_000.png


In [155]:
df_one_class.describe()

,Finding Labels,Image Index
count,91324,91324
unique,15,91324
top,No Finding,00022481_006.png
freq,60361,1


In [157]:
df_one_class['Finding Labels'].value_counts()

No Finding            60361
Infiltration           9547
Atelectasis            4215
Effusion               3955
Nodule                 2705
Pneumothorax           2194
Mass                   2139
Consolidation          1310
Pleural_Thickening     1126
Cardiomegaly           1093
Emphysema               892
Fibrosis                727
Edema                   628
Pneumonia               322
Hernia                  110
Name: Finding Labels, dtype: int64

In [176]:
class_list = df_one_class['Finding Labels'].unique().tolist()
class_list

['Cardiomegaly',
 'No Finding',
 'Hernia',
 'Infiltration',
 'Nodule',
 'Emphysema',
 'Effusion',
 'Atelectasis',
 'Pleural_Thickening',
 'Pneumothorax',
 'Mass',
 'Fibrosis',
 'Consolidation',
 'Edema',
 'Pneumonia']

In [0]:
test_list = ['test', 'test1']

In [182]:
for dirName in class_list:
  if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

Directory  Pneumonia  already exists


In [183]:
ls

 Atelectasis/          Hernia/       images_008/    'No Finding'/
 Cardiomegaly/         images_001/   images_009/     Pleural_Thickening/
 Consolidation/        images_002/   images_010/     Pneumonia/
 Data_Entry_2017.csv   images_003/   images_011/     Pneumothorax/
 Edema/                images_004/   images_012/     README_CHESTXRAY.pdf
 Effusion/             images_005/   Infiltration/
 Emphysema/            images_006/   Mass/
 Fibrosis/             images_007/   Nodule/


### 5) Copy images from directories into single class target directory  

In [200]:
count = 0
while count < 5 :
  #print(df_one_class.iloc[count][0], "\t", df_one_class.iloc[count][1])
  # CREATE A COPY OR A MOVE FROM SOURCE TO TARGET
  # SOURCE = images_001 OR SIMILAR FOLDER
  # TARGET = FOLDER WITH 'Finding Label' NAME
  count+=1


Cardiomegaly 	 00000001_000.png
No Finding 	 00000002_000.png
Hernia 	 00000003_000.png
Hernia 	 00000003_001.png
Hernia 	 00000003_002.png


### 6) Consider removing or compressing original image file directories  


### 7) Explore data, identify if need to adjust prior to tensors  

### 8) Create tensors


### 9) Remove directories or zip directories ...


In [185]:
for dirName in class_list:
  if not os.path.exists(dirName):
    print("Directory " , dirName ,  " Does not exist ")
  else:
    #os.rmdir(dirName)
    print("Directory " , dirName ,  " Removed ")

Directory  Cardiomegaly  Does not exist 
Directory  No Finding  Does not exist 
Directory  Hernia  Does not exist 
Directory  Infiltration  Does not exist 
Directory  Nodule  Does not exist 
Directory  Emphysema  Does not exist 
Directory  Effusion  Does not exist 
Directory  Atelectasis  Does not exist 
Directory  Pleural_Thickening  Does not exist 
Directory  Pneumothorax  Does not exist 
Directory  Mass  Does not exist 
Directory  Fibrosis  Does not exist 
Directory  Consolidation  Does not exist 
Directory  Edema  Does not exist 
Directory  Pneumonia  Does not exist 
